In [1]:
import matplotlib.pyplot as plt
import numpy as np
import SSLimPy.interface.sslimpy as sslimpy

import astropy.units as u

In [2]:
cosmodict={"h":0.67,"Omegam":0.32,"Omegab":0.04,"As":2.1e-9, "mnu":0.06}
settings = {"code":"class"}

Asslimpy = sslimpy.sslimpy(settings_dict=settings,
                           cosmopars=cosmodict)

#--------------------------------------------------#

  SSSSS   SSSSS  L       i            PPPP   y    y 
 S     S S     S L            m   m   P    P y   y  
 S       S       L     iii   m m m m  P    P  y y   
  SSSSS   SSSSS  L       i   m  m  m  PPPP     y    
       S       S L       i   m     m  P        y    
 S     S S     S L       i   m     m  P       y     
  SSSSS   SSSSS  LLLLL iiiii m     m  P      y      

#--------------------------------------------------#

----------CLASS Parameters--------

   k_per_decade_for_bao: 50
   k_per_decade_for_pk: 50
   P_k_max_1/Mpc: 50
   output: mPk,mTk
   non linear: halofit
   z_max_pk: 4.0
   N_ncdm: 1
   deg_ncdm: 1
   T_cmb: 2.7255
   nonlinear_min_k_max: 80.0
   halofit_tol_sigma: 1e-08
   k_pivot: 0.05
   YHe: 0.2454006
   h: 0.67
   n_s: 0.96
   A_s: 2.1e-09
   N_ur: 2.0286666666666666
   T_ncdm: 0.7163098828811857
   Omega_ncdm: 0.0014361044376033888
   Omega_b: 0.04
   Omega_cdm: 0.27856389556239664


----------RECAP OF SELEC